In [67]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import date
from datetime import timedelta
from datetime import datetime

%run web_scraping.ipynb

In [81]:
try:
    shows = pd.read_pickle("../data/shows.pkl")
except:
    shows = pd.DataFrame()
    
try:
    show_broadcast_details = pd.read_pickle("../data/show_broadcast_details.pkl")
except:
    show_broadcast_details = pd.DataFrame()
    
try:
    broadcast_details = pd.read_pickle("../data/broadcast_details.pkl")
except:
    broadcast_details = pd.DataFrame()
    
try:
    broadcast_playlists = pd.read_pickle("../data/broadcast_playlists.pkl")
except:
    broadcast_playlists = pd.DataFrame()

In [82]:
shows_to_run = ["https://www.bbc.co.uk/programmes/m00008zy/episodes/guide"]#, "https://www.bbc.co.uk/programmes/b0072lb2/episodes/guide"]

def allbut(levels, names):
    names = set(names)
    return [item for item in levels if item not in names]

temp_show_broadcast_details = pd.DataFrame()

for show_url in shows_to_run:
    show_instance = Show(show_url)
    if shows.empty or show_instance.show_key not in shows['Show Key'].unique():
        shows = pd.concat([shows,show_instance.show_details])
    temp_show_broadcast_details = pd.concat([temp_show_broadcast_details,show_instance.broadcasts])

In [83]:
temp_show_broadcast_details['Datetime Key'] = datetime.now()
temp_show_broadcast_details['Is New Data'] = True

show_broadcast_details = pd.concat([show_broadcast_details,temp_show_broadcast_details])
show_broadcast_details = show_broadcast_details.groupby(allbut(show_broadcast_details.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
show_broadcast_details = show_broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

In [84]:
new_broadcast_data = show_broadcast_details[(show_broadcast_details["Is Broadcast Upcoming"]==False) & (show_broadcast_details["Is New Data"]==True)][0:10]
temp_broadcast_details = pd.DataFrame()
temp_broadcast_playlists = pd.DataFrame()

for i, [broadcast_url,broadcast_key] in enumerate(new_broadcast_data[['Broadcast URL','Broadcast Key']].values):
    broadcast_instance = Playlist(broadcast_url)
    print(f'Broadcast Number {1+i} of {len(new_broadcast_data.index)}')
    temp_broadcast_details = pd.concat([temp_broadcast_details,broadcast_instance.playlist])
    temp_broadcast_playlists = pd.concat([temp_broadcast_playlists,broadcast_instance.playlist])
    
temp_broadcast_details['Datetime Key'] = datetime.now()
temp_broadcast_details['Is New Data'] = True
temp_broadcast_playlists['Datetime Key'] = datetime.now()
temp_broadcast_playlists['Is New Data'] = True

broadcast_details = pd.concat([broadcast_details,temp_broadcast_details])
broadcast_details = broadcast_details.groupby(allbut(broadcast_details.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_details = broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

broadcast_playlists = pd.concat([broadcast_playlists,temp_broadcast_playlists])
broadcast_playlists = broadcast_playlists.groupby(allbut(broadcast_playlists.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_playlists = broadcast_playlists.groupby(["Broadcast Key", "Broadcast Position"]).apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)


In [72]:
show_broadcast_details['Is New Data'] = False
broadcast_details['Is New Data'] = False
broadcast_playlists['Is New Data'] = False

shows.to_pickle("../data/shows.pkl")
show_broadcast_details.to_pickle("../data/show_broadcast_details.pkl")
broadcast_details.to_pickle("../data/broadcast_details.pkl")
broadcast_playlists.to_pickle("../data/broadcast_playlists.pkl")

In [88]:
show_broadcast_details

,Broadcast Image URL,Broadcast URL,Broadcast Title,Broadcast Short Description,Is Broadcast Upcoming,Broadcast Key,Show Key,Datetime Key,Is New Data
0,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m000cmn2,Next Wave 2019,Jack picks his favourite discoveries from Next...,False,-9135362881644799146,-2655158488449224554,2022-12-10 15:45:15.350767,False
1,https://ichef.bbci.co.uk/images/ic/320x180/p07...,https://www.bbc.co.uk/programmes/m00050gj,The Indie House Party with Mini Mansions!,Jack and Mini Mansions ramp up the Radio 1 wee...,False,-9093479655164975711,-2655158488449224554,2022-12-10 15:45:15.350767,False
2,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m0008hy3,Battle of the Fans!,The search to find the ultimate indie fan cont...,False,-9071923286495823111,-2655158488449224554,2022-12-10 15:45:15.350767,False
3,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m0005p9z,Battle of the Fans!,The search to find the ultimate indie fan cont...,False,-9046601813021399571,-2655158488449224554,2022-12-10 15:45:15.350767,False
4,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m0002f73,Battle of the Fans!,The search to find the ultimate indie fan cont...,False,-9018781428427033947,-2655158488449224554,2022-12-10 15:45:15.350767,False
...,...,...,...,...,...,...,...,...,...
531,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m0015c74,Sundara Karma,Oscar from Sundara Karma joins Jack for the In...,False,9098165914593094156,-2655158488449224554,2022-12-10 15:45:15.350767,False
532,https://ichef.bbci.co.uk/images/ic/320x180/p06...,https://www.bbc.co.uk/programmes/m0002gjk,Thursday Night House Party with Scott Mills an...,"Jack, Scott and Chris get the Radio 1 weekend ...",False,9116864633646694223,-2655158488449224554,2022-12-10 15:45:15.350767,False
533,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m00169bc,Blossoms,Tom and Joe from Blossoms join Jack for the In...,False,9168535692854716338,-2655158488449224554,2022-12-10 15:45:15.350767,False
534,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m000cnnl,Mince Pies with Jack Saunders!,Jack invites some special guests into the stud...,False,9169135045801013394,-2655158488449224554,2022-12-10 15:45:15.350767,False
